**Data Loading and quick clean/check**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load the datasets
products_df = pd.read_csv('/Fetch - Senior Data Analyst/PRODUCTS_TAKEHOME (1).csv')
transactions_df = pd.read_csv('/Fetch - Senior Data Analyst/TRANSACTION_TAKEHOME (1).csv')
users_df = pd.read_csv('/Fetch - Senior Data Analyst/USER_TAKEHOME (1).csv')

# Convert date columns to datetime
users_df['CREATED_DATE'] = pd.to_datetime(users_df['CREATED_DATE'])
users_df['BIRTH_DATE'] = pd.to_datetime(users_df['BIRTH_DATE'])
transactions_df['PURCHASE_DATE'] = pd.to_datetime(transactions_df['PURCHASE_DATE'])
transactions_df['SCAN_DATE'] = pd.to_datetime(transactions_df['SCAN_DATE'])

# Convert FINAL_SALE to numeric (may need adjustment based on your data)
transactions_df['FINAL_SALE'] = pd.to_numeric(transactions_df['FINAL_SALE'], errors='coerce')

# Remove duplicates
transactions_df = transactions_df.drop_duplicates()
products_df = products_df.drop_duplicates()

# Convert date columns to datetime, ensuring consistent timezone handling
for col in ['PURCHASE_DATE', 'SCAN_DATE']:
    if col in transactions_df.columns:
        transactions_df[col] = pd.to_datetime(transactions_df[col], utc=True).dt.tz_localize(None)

for col in ['CREATED_DATE', 'BIRTH_DATE']:
    if col in users_df.columns:
        users_df[col] = pd.to_datetime(users_df[col], utc=True).dt.tz_localize(None)

# Use a timezone-naive current date
current_date = pd.Timestamp.now().tz_localize(None)

# Print basic info about datasets
print(f"Users: {users_df.shape}, Transactions: {transactions_df.shape}, Products: {products_df.shape}")

Users: (100000, 6), Transactions: (49829, 8), Products: (845337, 7)


**Part 1: Explore the Data Quality Issues**

In [2]:
# Identify data quality issues
print("\n=== DATA QUALITY ISSUES ===")

# Missing values across datasets
missing_users = users_df.isnull().sum()
missing_transactions = transactions_df.isnull().sum()
missing_products = products_df.isnull().sum()

print("\nMissing values in key columns:")
print(f"Users - BIRTH_DATE: {missing_users['BIRTH_DATE']} ({missing_users['BIRTH_DATE']/len(users_df)*100:.1f}%)")
print(f"Transactions - BARCODE: {missing_transactions['BARCODE']} ({missing_transactions['BARCODE']/len(transactions_df)*100:.1f}%)")
print(f"Products - BRAND: {missing_products['BRAND']} ({missing_products['BRAND']/len(products_df)*100:.1f}%)")

# Check for negative scan delays
transactions_df['SCAN_DELAY'] = (transactions_df['SCAN_DATE'] - transactions_df['PURCHASE_DATE']).dt.days
neg_scan_delays = (transactions_df['SCAN_DELAY'] < 0).sum()
print(f"\nNegative scan delays: {neg_scan_delays} ({neg_scan_delays/len(transactions_df)*100:.1f}%)")

# Check for invalid FINAL_QUANTITY values
print(f"\nUnique FINAL_QUANTITY values: {transactions_df['FINAL_QUANTITY'].nunique()}")
print(transactions_df['FINAL_QUANTITY'].value_counts().head())

# Check for potential barcode issues
invalid_barcodes = (products_df['BARCODE'] <= 0).count()
print(f"\nInvalid barcodes (<=0): {invalid_barcodes}")

# Data consistency check
user_ids_in_transactions = transactions_df['USER_ID'].nunique()
user_ids_in_users = users_df['ID'].nunique()
print(f"\nUser IDs in transactions: {user_ids_in_transactions}")
print(f"User IDs in users table: {user_ids_in_users}")
print(f"Difference: {user_ids_in_users - user_ids_in_transactions}")

# Fields that might be challenging to understand
print("\n=== CHALLENGING FIELDS ===")
print("1. FINAL_QUANTITY has non-numeric values like 'zero'")
print("2. Multiple category levels make analysis complex")
print("3. Some barcodes are missing, affecting product identification")
print("4. Negative scan delays suggest data quality issues")


=== DATA QUALITY ISSUES ===

Missing values in key columns:
Users - BIRTH_DATE: 3675 (3.7%)
Transactions - BARCODE: 5735 (11.5%)
Products - BRAND: 226462 (26.8%)

Negative scan delays: 94 (0.2%)

Unique FINAL_QUANTITY values: 87
FINAL_QUANTITY
1.00    35536
zero    12491
2.00     1285
3.00      184
4.00      139
Name: count, dtype: int64

Invalid barcodes (<=0): 845337

User IDs in transactions: 17694
User IDs in users table: 100000
Difference: 82306

=== CHALLENGING FIELDS ===
1. FINAL_QUANTITY has non-numeric values like 'zero'
2. Multiple category levels make analysis complex
3. Some barcodes are missing, affecting product identification
4. Negative scan delays suggest data quality issues


In [3]:
# ====== DATA CLEANING ======
print("\n=== DATA CLEANING ===")

# Make a copy of original dataframes for cleaning
users_clean = users_df.copy()
transactions_clean = transactions_df.copy()
products_clean = products_df.copy()

# 1. CLEAN USERS DATA
print("Cleaning users data...")

# Handle missing birth dates
# Approach: We'll keep records with missing birth dates but flag them
users_clean['HAS_BIRTHDATE'] = ~users_clean['BIRTH_DATE'].isna()

# Check for unrealistic birth dates (users younger than 13 or older than 100)
current_date = pd.Timestamp.now()
users_clean['AGE'] = (current_date - users_clean['BIRTH_DATE']).dt.days / 365.25
unrealistic_ages = ((users_clean['AGE'] < 13) | (users_clean['AGE'] > 100)).sum()
print(f"  Found {unrealistic_ages} users with unrealistic ages (< 13 or > 100)")

# Flag unrealistic ages instead of removing them
users_clean['VALID_AGE'] = ~((users_clean['AGE'] < 13) | (users_clean['AGE'] > 100))

# 2. CLEAN TRANSACTIONS DATA
print("Cleaning transactions data...")

# Remove duplicates
dup_count = transactions_clean.duplicated().sum()
transactions_clean = transactions_clean.drop_duplicates()
print(f"  Removed {dup_count} duplicate transactions")

# Convert FINAL_QUANTITY to numeric
# First, replace "zero" with "0"
transactions_clean['FINAL_QUANTITY'] = transactions_clean['FINAL_QUANTITY'].replace('zero', '0')
# Convert to numeric, coercing errors to NaN
transactions_clean['FINAL_QUANTITY'] = pd.to_numeric(transactions_clean['FINAL_QUANTITY'], errors='coerce')
print(f"  Converted FINAL_QUANTITY to numeric format")

# Convert FINAL_SALE to numeric
transactions_clean['FINAL_SALE'] = pd.to_numeric(transactions_clean['FINAL_SALE'], errors='coerce')
missing_sales = transactions_clean['FINAL_SALE'].isna().sum()
print(f"  Found {missing_sales} transactions with missing sale values")

# For transactions with missing sale values, impute with median for the same barcode
median_sales = transactions_clean.groupby('BARCODE')['FINAL_SALE'].transform('median')
transactions_clean.loc[transactions_clean['FINAL_SALE'].isna(), 'FINAL_SALE'] = median_sales
still_missing = transactions_clean['FINAL_SALE'].isna().sum()

# For any remaining missing sales (where we couldn't impute from same barcode), use overall median
if still_missing > 0:
    overall_median = transactions_clean['FINAL_SALE'].median()
    transactions_clean['FINAL_SALE'].fillna(overall_median, inplace=True)
    print(f"  Imputed {missing_sales - still_missing} missing sales with barcode median")
    print(f"  Imputed {still_missing} remaining missing sales with overall median")

# Fix negative scan delays
transactions_clean['SCAN_DELAY'] = (transactions_clean['SCAN_DATE'] - transactions_clean['PURCHASE_DATE']).dt.days
neg_delays = (transactions_clean['SCAN_DELAY'] < 0).sum()
if neg_delays > 0:
    print(f"  Found {neg_delays} transactions with negative scan delays")
    # Swap purchase and scan dates for transactions with negative delays
    neg_delay_mask = transactions_clean['SCAN_DELAY'] < 0
    temp_purchase = transactions_clean.loc[neg_delay_mask, 'PURCHASE_DATE'].copy()
    transactions_clean.loc[neg_delay_mask, 'PURCHASE_DATE'] = transactions_clean.loc[neg_delay_mask, 'SCAN_DATE']
    transactions_clean.loc[neg_delay_mask, 'SCAN_DATE'] = temp_purchase
    # Recalculate scan delay
    transactions_clean['SCAN_DELAY'] = (transactions_clean['SCAN_DATE'] - transactions_clean['PURCHASE_DATE']).dt.days
    print(f"  Fixed negative scan delays by swapping purchase and scan dates")

# 3. CLEAN PRODUCTS DATA
print("Cleaning products data...")

# Remove duplicates
dup_count = products_clean.duplicated().sum()
products_clean = products_clean.drop_duplicates()
print(f"  Removed {dup_count} duplicate products")

# Handle missing brand information
missing_brands = products_clean['BRAND'].isna().sum()
if missing_brands > 0:
    print(f"  Found {missing_brands} products with missing brand information")
    # Fill missing brands with manufacturer where available
    mask = (products_clean['BRAND'].isna()) & (~products_clean['MANUFACTURER'].isna())
    products_clean.loc[mask, 'BRAND'] = products_clean.loc[mask, 'MANUFACTURER']
    filled_count = mask.sum()
    print(f"  Filled {filled_count} missing brands with manufacturer name")

    # For remaining missing brands, fill with "UNKNOWN"
    still_missing = products_clean['BRAND'].isna().sum()
    if still_missing > 0:
        products_clean['BRAND'].fillna('UNKNOWN', inplace=True)
        print(f"  Filled {still_missing} remaining missing brands with 'UNKNOWN'")

# Handle invalid barcodes
invalid_barcodes = (products_clean['BARCODE'] <= 0).sum()
if invalid_barcodes > 0:
    print(f"  Found {invalid_barcodes} products with invalid barcodes (<=0)")
    # We'll flag these but not remove them
    products_clean['VALID_BARCODE'] = products_clean['BARCODE'] > 0

# After cleaning, report the dimensions of the cleaned datasets
print("\nCleaned dataset dimensions:")
print(f"Users: {users_clean.shape} (original: {users_df.shape})")
print(f"Transactions: {transactions_clean.shape} (original: {transactions_df.shape})")
print(f"Products: {products_clean.shape} (original: {products_df.shape})")


# joined dataframes with cleaned data
transactions_with_products = transactions_clean.merge(products_clean, on='BARCODE', how='left')
full_data = transactions_with_products.merge(users_clean, left_on='USER_ID', right_on='ID', how='left')


=== DATA CLEANING ===
Cleaning users data...
  Found 109 users with unrealistic ages (< 13 or > 100)
Cleaning transactions data...
  Removed 0 duplicate transactions
  Converted FINAL_QUANTITY to numeric format
  Found 12486 transactions with missing sale values
  Imputed 11031 missing sales with barcode median
  Imputed 1455 remaining missing sales with overall median
  Found 94 transactions with negative scan delays
  Fixed negative scan delays by swapping purchase and scan dates
Cleaning products data...
  Removed 0 duplicate products
  Found 226462 products with missing brand information
  Filled 0 missing brands with manufacturer name
  Filled 226462 remaining missing brands with 'UNKNOWN'

Cleaned dataset dimensions:
Users: (100000, 9) (original: (100000, 6))
Transactions: (49829, 9) (original: (49829, 9))
Products: (845337, 7) (original: (845337, 7))


**Part 2: SQL Queries (Writing in Python)**

In [5]:
print("\n=== SQL QUERIES ===")

# Question 1: Top 5 brands by receipts scanned among users 21 and over
print("\n--- Question 1: Top 5 brands by receipts scanned among users 21 and over ---")

# Calculate age
users_clean['AGE'] = (current_date - users_clean['BIRTH_DATE']).dt.days / 365.25
users_21_plus = users_clean[users_clean['AGE'] >= 21]

# SQL equivalent:
# SELECT p.BRAND, COUNT(DISTINCT t.RECEIPT_ID) as receipt_count
# FROM transactions t
# JOIN users u ON t.USER_ID = u.ID
# JOIN products p ON t.BARCODE = p.BARCODE
# WHERE (CURRENT_DATE - u.BIRTH_DATE)/365.25 >= 21
# GROUP BY p.BRAND
# ORDER BY receipt_count DESC
# LIMIT 5;

# Python implementation
brands_21plus = transactions_clean.merge(
    users_21_plus[['ID']],
    left_on='USER_ID',
    right_on='ID',
    how='inner'
)
brands_21plus = brands_21plus.merge(
    products_clean[['BARCODE', 'BRAND']],
    on='BARCODE',
    how='inner'
)
top_brands_receipts = brands_21plus.groupby('BRAND')['RECEIPT_ID'].nunique().reset_index()
top_brands_receipts = top_brands_receipts.sort_values('RECEIPT_ID', ascending=False).head(5)
print(top_brands_receipts)

# Question 2: Top 5 brands by sales among users that have had their account for at least six months
print("\n--- Question 2: Top 5 brands by sales among users with 6+ month accounts ---")

# SQL equivalent:
# SELECT p.BRAND, SUM(t.FINAL_SALE) as total_sales
# FROM transactions t
# JOIN users u ON t.USER_ID = u.ID
# JOIN products p ON t.BARCODE = p.BARCODE
# WHERE u.CREATED_DATE <= DATE_SUB(CURRENT_DATE, INTERVAL 6 MONTH)
# GROUP BY p.BRAND
# ORDER BY total_sales DESC
# LIMIT 5;

# Calculate 6 months ago
six_months_ago = current_date - pd.DateOffset(months=6)
users_6months = users_clean[users_df['CREATED_DATE'] <= six_months_ago]

# Python implementation
brands_6months = transactions_clean.merge(
    users_6months[['ID']],
    left_on='USER_ID',
    right_on='ID',
    how='inner'
)
brands_6months = brands_6months.merge(
    products_clean[['BARCODE', 'BRAND']],
    on='BARCODE',
    how='inner'
)
top_brands_sales = brands_6months.groupby('BRAND')['FINAL_SALE'].sum().reset_index()
top_brands_sales = top_brands_sales.sort_values('FINAL_SALE', ascending=False).head(5)
print(top_brands_sales)

# Question 3: Percentage of sales in the Health & Wellness category by generation
print("\n--- Question 3: Percentage of sales in Health & Wellness category ---")

# SQL equivalent:
# WITH health_wellness_sales AS (
#   SELECT SUM(t.FINAL_SALE) as hw_sales
#   FROM transactions t
#   JOIN products p ON t.BARCODE = p.BARCODE
#   WHERE p.CATEGORY_1 = 'Health & Wellness'
# ),
# total_sales AS (
#   SELECT SUM(t.FINAL_SALE) as total_sales
#   FROM transactions t
# )
# SELECT (hw_sales / total_sales) * 100 as percentage
# FROM health_wellness_sales, total_sales;

# Python implementation
# First, join transactions with products
txn_products = transactions_clean.merge(
    products_clean,
    on='BARCODE',
    how='inner'
)

# Calculate total sales in Health & Wellness
hw_sales = txn_products[txn_products['CATEGORY_1'] == 'Health & Wellness']['FINAL_SALE'].sum()
total_sales = txn_products['FINAL_SALE'].sum()

hw_percentage = (hw_sales / total_sales) * 100 if total_sales > 0 else 0
print(f"Percentage of sales in Health & Wellness: {hw_percentage:.2f}%")


=== SQL QUERIES ===

--- Question 1: Top 5 brands by receipts scanned among users 21 and over ---
               BRAND  RECEIPT_ID
204             DOVE          14
746          UNKNOWN          14
491      NERDS CANDY          14
164        COCA-COLA          13
674  SOUR PATCH KIDS          13

--- Question 2: Top 5 brands by sales among users with 6+ month accounts ---
                         BRAND  FINAL_SALE
746                    UNKNOWN   26870.710
37   ANNIE'S HOMEGROWN GROCERY    2609.040
204                       DOVE    2562.415
69                    BAREFOOT    2500.330
535                      ORIBE    2282.910

--- Question 3: Percentage of sales in Health & Wellness category ---
Percentage of sales in Health & Wellness: 57.89%


**Part 3: Open-ended Questions**

In [6]:
print("\n=== OPEN-ENDED QUESTIONS ===")

# Question 4: Who are Fetch's power users?
print("\n--- Who are Fetch's power users? ---")

# Calculate metrics per user
user_metrics = transactions_clean.groupby('USER_ID').agg(
    receipt_count=('RECEIPT_ID', 'nunique'),
    purchase_count=('RECEIPT_ID', 'count'),
    total_spend=('FINAL_SALE', 'sum'),
    avg_time_to_scan=('SCAN_DELAY', 'mean'),
    stores_visited=('STORE_NAME', 'nunique')
).reset_index()

# Define power users (top 5% in receipt count and total spend)
receipt_threshold = np.percentile(user_metrics['receipt_count'], 95)
spend_threshold = np.percentile(user_metrics['total_spend'], 95)

power_users = user_metrics[
    (user_metrics['receipt_count'] >= receipt_threshold) &
    (user_metrics['total_spend'] >= spend_threshold)
]

# Get demographic information
power_users_demo = power_users.merge(users_clean, left_on='USER_ID', right_on='ID', how='left')

print(f"Number of power users: {len(power_users)} ({len(power_users)/len(user_metrics)*100:.2f}% of user base)")
print("\nPower user metrics:")
print(f"Average receipts: {power_users['receipt_count'].mean():.1f}")
print(f"Average spend: ${power_users['total_spend'].mean():.2f}")
print(f"Average stores visited: {power_users['stores_visited'].mean():.1f}")

# State distribution of power users
if 'STATE' in power_users_demo.columns:
    top_states = power_users_demo['STATE'].value_counts().head(5)
    print("\nTop states for power users:")
    for state, count in top_states.items():
        print(f"{state}: {count}")

# Question 5: Which is the leading brand in the Dips & Salsa category?
print("\n--- Which is the leading brand in the Dips & Salsa category? ---")

# Find products in Dips & Salsa category
dips_salsa_products = products_clean[
    (products_clean['CATEGORY_1'] == 'Dips & Salsa') |
    (products_clean['CATEGORY_2'] == 'Dips & Salsa') |
    (products_clean['CATEGORY_3'] == 'Dips & Salsa') |
    (products_clean['CATEGORY_4'] == 'Dips & Salsa')
]

if len(dips_salsa_products) > 0:
    # Get sales for these products
    dips_salsa_sales = transactions_clean.merge(
        dips_salsa_products[['BARCODE', 'BRAND']],
        on='BARCODE',
        how='inner'
    )

    brand_sales = dips_salsa_sales.groupby('BRAND')['FINAL_SALE'].sum().reset_index()
    leading_brand = brand_sales.sort_values('FINAL_SALE', ascending=False).iloc[0]

    print(f"Leading brand in Dips & Salsa: {leading_brand['BRAND']} (${leading_brand['FINAL_SALE']:.2f} in sales)")

    # Show market share
    total_dips_sales = brand_sales['FINAL_SALE'].sum()
    brand_sales['MARKET_SHARE'] = brand_sales['FINAL_SALE'] / total_dips_sales * 100

    print("\nMarket share of top 3 brands:")
    for _, row in brand_sales.sort_values('FINAL_SALE', ascending=False).head(3).iterrows():
        print(f"{row['BRAND']}: {row['MARKET_SHARE']:.1f}%")
else:
    # Look for categories that might contain dips or salsa
    print("No explicit 'Dips & Salsa' category found. Searching for related products...")
    dips_related = products_clean[
        products_clean['CATEGORY_1'].str.contains('Dips|Salsa', na=False, case=False) |
        products_clean['CATEGORY_2'].str.contains('Dips|Salsa', na=False, case=False) |
        products_clean['CATEGORY_3'].str.contains('Dips|Salsa', na=False, case=False)
    ]
    print(f"Found {len(dips_related)} potentially related products")

# Question 6: At what percent has Fetch grown year over year?
print("\n--- At what percent has Fetch grown year over year? ---")

# Add year to transactions
transactions_clean['YEAR'] = transactions_clean['PURCHASE_DATE'].dt.year
transactions_clean['MONTH'] = transactions_clean['PURCHASE_DATE'].dt.month

# Calculate monthly metrics
monthly_metrics = transactions_clean.groupby(['YEAR', 'MONTH']).agg(
    receipts=('RECEIPT_ID', 'nunique'),
    active_users=('USER_ID', 'nunique'),
    total_spend=('FINAL_SALE', 'sum')
).reset_index()

# Calculate year over year growth
if monthly_metrics['YEAR'].nunique() > 1:
    yearly_metrics = transactions_df.groupby('YEAR').agg(
        receipts=('RECEIPT_ID', 'nunique'),
        active_users=('USER_ID', 'nunique'),
        total_spend=('FINAL_SALE', 'sum')
    ).reset_index()

    yearly_metrics['receipt_growth'] = yearly_metrics['receipts'].pct_change() * 100
    yearly_metrics['user_growth'] = yearly_metrics['active_users'].pct_change() * 100
    yearly_metrics['spend_growth'] = yearly_metrics['total_spend'].pct_change() * 100

    print(yearly_metrics)
else:
    print("Data only contains transactions from one year (2024). Cannot calculate YoY growth.")
    print("Showing monthly growth instead:")

    monthly_metrics['MOM_user_growth'] = monthly_metrics.groupby('YEAR')['active_users'].pct_change() * 100
    print(monthly_metrics[['YEAR', 'MONTH', 'active_users', 'MOM_user_growth']].tail())


=== OPEN-ENDED QUESTIONS ===

--- Who are Fetch's power users? ---
Number of power users: 305 (1.72% of user base)

Power user metrics:
Average receipts: 3.7
Average spend: $58.03
Average stores visited: 2.2

Top states for power users:
NY: 1

--- Which is the leading brand in the Dips & Salsa category? ---
Leading brand in Dips & Salsa: UNKNOWN ($274471.00 in sales)

Market share of top 3 brands:
UNKNOWN: 11.3%
TOSTITOS: 5.2%
FRITOS: 4.1%

--- At what percent has Fetch grown year over year? ---
Data only contains transactions from one year (2024). Cannot calculate YoY growth.
Showing monthly growth instead:
   YEAR  MONTH  active_users  MOM_user_growth
0  2024      6          4948              NaN
1  2024      7          7957        60.812449
2  2024      8          7373        -7.339450
3  2024      9          1596       -78.353452


**Part 4: Visualizations**

In [10]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Top 5 Brands by Receipts Visualization
fig_brands = px.bar(
    top_brands_receipts,
    x='BRAND',
    y='RECEIPT_ID',
    title='Top 5 Brands by Receipts (Users 21+)',
    labels={'BRAND': 'Brand', 'RECEIPT_ID': 'Number of Receipts'},
    text='RECEIPT_ID',  # Show values on bars
    color_discrete_sequence=px.colors.qualitative.Set1  # Use distinct colors
)

# Customize layout
fig_brands.update_layout(
    xaxis_title='Brand',
    yaxis_title='Number of Receipts',
    xaxis_tickangle=-45,
    height=600,
    width=900,
    margin=dict(l=50, r=50, t=80, b=100),
    plot_bgcolor='white',  # White background
    font=dict(size=12)
)

# Format text display
fig_brands.update_traces(texttemplate='%{text:,}', textposition='outside')

# Show the plot
fig_brands.show()

# Sales distribution by category
category_sales = txn_products.groupby('CATEGORY_1')['FINAL_SALE'].sum().reset_index()
category_sales = category_sales.sort_values('FINAL_SALE', ascending=False).head(10)

fig_categories = px.bar(
    category_sales,
    x='CATEGORY_1',
    y='FINAL_SALE',
    title='Top 10 Categories by Sales',
    labels={'CATEGORY_1': 'Category', 'FINAL_SALE': 'Total Sales ($)'},
    text='FINAL_SALE',  # Show values on bars
    color='CATEGORY_1',  # Each category gets its own color
    color_discrete_sequence=px.colors.qualitative.Set2  # Using Set2 instead of Paired
)

# Customize layout
fig_categories.update_layout(
    xaxis_title='Category',
    yaxis_title='Total Sales ($)',
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    margin=dict(l=50, r=50, t=80, b=100),
    plot_bgcolor='white',  # White background
    showlegend=False,  # Hide legend since x-axis shows categories
    font=dict(size=12)
)

# Format y-axis to show currency and text display
fig_categories.update_yaxes(tickprefix='$')
fig_categories.update_traces(texttemplate='$%{text:,.2f}', textposition='outside')

# Show the plot
fig_categories.show()

# BONUS: Interactive dashboard with both charts
# Create a subplot with 1 row and 2 columns
fig_dashboard = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=('<b>Top 5 Brands by Receipts (Users 21+)</b>', '<b>Top 10 Categories by Sales</b>'),
    vertical_spacing=0.3,
    specs=[[{"type": "bar"}], [{"type": "bar"}]]
)

# Create color scales for consistency
brand_colors = px.colors.qualitative.Set3[:len(top_brands_receipts)]  # Using Set3 instead of Bold
category_colors = px.colors.qualitative.Set2[:len(category_sales)]     # Using Set2 instead of Pastel

# Add first bar chart
fig_dashboard.add_trace(
    go.Bar(
        x=top_brands_receipts['BRAND'],
        y=top_brands_receipts['RECEIPT_ID'],
        marker_color=brand_colors,
        text=top_brands_receipts['RECEIPT_ID'],
        textposition='outside',
        texttemplate='%{text:,}'
    ),
    row=1, col=1
)

# Add second bar chart
fig_dashboard.add_trace(
    go.Bar(
        x=category_sales['CATEGORY_1'],
        y=category_sales['FINAL_SALE'],
        marker_color=category_colors,
        text=category_sales['FINAL_SALE'],
        textposition='outside',
        texttemplate='$%{text:,.2f}'
    ),
    row=2, col=1
)

# Update layout
fig_dashboard.update_layout(
    height=1000,  # Increased height for better spacing
    width=1000,
    showlegend=False,
    title_text="<b>Fetch Data Analysis Dashboard</b>",
    title_font_size=20,
    font=dict(family="Arial, sans-serif", size=12),
    plot_bgcolor='white',  # White background
    paper_bgcolor='white'  # White paper background
)

# Update axes
fig_dashboard.update_xaxes(tickangle=-45, row=1, col=1, title_text="Brand")
fig_dashboard.update_xaxes(tickangle=-45, row=2, col=1, title_text="Category")
fig_dashboard.update_yaxes(title_text="Number of Receipts", row=1, col=1, gridcolor='lightgray')
fig_dashboard.update_yaxes(title_text="Total Sales ($)", tickprefix='$', row=2, col=1, gridcolor='lightgray')

# Add hover templates for better tooltips
fig_dashboard.update_traces(
    hovertemplate="<b>%{x}</b><br>Number: %{y:,}<extra></extra>",
    row=1, col=1
)
fig_dashboard.update_traces(
    hovertemplate="<b>%{x}</b><br>Sales: $%{y:,.2f}<extra></extra>",
    row=2, col=1
)

# Show dashboard
fig_dashboard.show()

In [13]:
import os
if not os.path.exists('visualizations'):
    os.makedirs('visualizations')
fig_dashboard.write_html('visualizations/dashboard.html')